In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze;

In [0]:
import pandas as pd

docs = [
    'academic.csv',
    'applications.csv',
    'loans.csv',
    'students.csv',
    'transactions_monthly.csv',
    'data_dictionary.csv',
    'repayments.csv'
]

path = []

for doc in docs:
  df = pd.read_csv(f'/Workspace/Users/carlos.e.garcia6@aluno.senai.br/DATABRICKS_CHALLENGE/{doc}')
  spark.createDataFrame(df).write.format('delta').mode('overwrite').saveAsTable(f'workspace.bronze.{doc.replace('.csv', '')}')
  path.append(f'workspace.bronze.{doc.replace('.csv', '')}')

path

['workspace.bronze.academic',
 'workspace.bronze.applications',
 'workspace.bronze.loans',
 'workspace.bronze.students',
 'workspace.bronze.transactions_monthly',
 'workspace.bronze.data_dictionary',
 'workspace.bronze.repayments']

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.students_clean AS
SELECT DISTINCT *
FROM workspace.bronze.students

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT DISTINCT count(*) FROM workspace.bronze.academic

count(*)
1800


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.consolidated_table_academic AS
  SELECT 
    a.* EXCEPT(student_id),
    b.*
  FROM workspace.silver.students_clean AS a
  LEFT JOIN (
    SELECT *
    FROM (
      SELECT *,
             ROW_NUMBER() OVER (PARTITION BY student_id ORDER BY CAST(last_update AS DATE) DESC) AS rn
      FROM workspace.bronze.academic
    )
    WHERE rn = 1
  ) AS b ON a.student_id = b.student_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.applications_clean AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY application_id
               ORDER BY application_date DESC
           ) AS rn
    FROM workspace.bronze.applications
) t
WHERE rn = 1

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM workspace.bronze.loans

loan_id,application_id,student_id,loan_amount,interest_rate_monthly,installments,start_date,status
L000739,A001204,S000696,5925.69,0.0182,24,2024-11-28,ACTIVE
L000740,A001205,S000856,3033.95,0.032,12,2024-07-29,CLOSED
L000741,A001206,S001068,1345.29,0.0195,24,2024-07-30,DEFAULT
L000742,A001207,S000356,1257.25,0.0198,6,2024-11-27,CLOSED
L000743,A001209,S000047,1301.37,0.0211,12,2024-07-04,ACTIVE
L000744,A001210,S000592,856.44,0.0152,12,2024-02-10,ACTIVE
L000745,A001211,S001643,1144.71,0.0193,3,2024-10-31,CLOSED
L000746,A001213,S001094,1690.49,0.011,9,2025-01-31,CLOSED
L000747,A001215,S001092,5119.0,0.0138,12,2024-06-25,ACTIVE
L000748,A001216,S000662,4723.56,0.0296,9,2024-07-11,ACTIVE


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.loans_agg AS
WITH ranked_loans AS (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY student_id
               ORDER BY start_date DESC
           ) AS rn
    FROM workspace.bronze.loans
)

SELECT
    student_id,

    -- Último status real baseado na data mais recente
    MAX(CASE WHEN rn = 1 THEN status END) AS last_loan_status,

    -- Já teve algum empréstimo em default?
    MAX(CASE WHEN status = 'default' THEN 1 ELSE 0 END) AS ever_default,

    COUNT(*) AS total_loans

FROM ranked_loans
GROUP BY student_id;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.transactions_agg AS
SELECT
    student_id,

    AVG(inflows) AS avg_inflows,
    AVG(outflows) AS avg_outflows,

    AVG(avg_balance) AS avg_balance,

    SUM(overdraft_events) AS total_overdraft_events,
    SUM(late_fees_count) AS total_late_fees,

    -- Métrica estratégica
    AVG(inflows - outflows) AS avg_net_cashflow

FROM workspace.bronze.transactions_monthly
GROUP BY student_id;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.consolidated_table_applications AS
SELECT 
    -- Dados do estudante
    s.student_id,
    s.birth_date,
    s.created_at,

    -- Dados da aplicação
    a.application_id,
    a.application_date,

    -- Dados agregados de crédito
    l.last_loan_status,
    l.ever_default,
    l.total_loans,

    -- Dados financeiros agregados
    t.avg_inflows,
    t.avg_outflows,
    t.avg_balance,
    t.total_overdraft_events,
    t.total_late_fees,
    t.avg_net_cashflow

FROM workspace.silver.students_clean AS s

LEFT JOIN workspace.silver.applications_clean AS a
    ON s.student_id = a.student_id

LEFT JOIN workspace.silver.loans_agg AS l
    ON s.student_id = l.student_id

LEFT JOIN workspace.silver.transactions_agg AS t
    ON s.student_id = t.student_id;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold.credit_policy_base AS
SELECT
    application_id,
    student_id,

    -- Capacidade mensal estimada
    avg_net_cashflow,

    -- Indicador de stress financeiro
    CASE 
        WHEN total_overdraft_events >= 5 THEN 'high'
        WHEN total_overdraft_events BETWEEN 2 AND 4 THEN 'medium'
        ELSE 'low'
    END AS overdraft_risk,

    -- Histórico negativo
    ever_default,

    -- Estabilidade financeira
    CASE
        WHEN avg_balance >= 5000 THEN 'high'
        WHEN avg_balance BETWEEN 1000 AND 4999 THEN 'medium'
        ELSE 'low'
    END AS balance_profile,

    total_loans,
    total_late_fees

FROM workspace.silver.consolidated_table_applications;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold.credit_risk_segmented AS
SELECT
    *,
    
    CASE
        WHEN ever_default = 1 THEN 'high_risk'

        WHEN overdraft_risk = 'high'
             AND avg_net_cashflow < 0 THEN 'high_risk'

        WHEN overdraft_risk = 'medium'
             OR avg_net_cashflow BETWEEN 0 AND 1000 THEN 'medium_risk'

        ELSE 'low_risk'
    END AS risk_segment

FROM workspace.gold.credit_policy_base;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold.final_credit_policy AS
SELECT
    application_id,
    student_id,
    risk_segment,

    CASE
        WHEN risk_segment = 'high_risk' THEN 'reject_or_require_guarantor'
        WHEN risk_segment = 'medium_risk' THEN 'approve_with_reduced_limit'
        ELSE 'approve'
    END AS recommended_action,

    CASE
        WHEN risk_segment = 'high_risk' THEN 0
        WHEN risk_segment = 'medium_risk' THEN avg_net_cashflow * 3
        ELSE avg_net_cashflow * 6
    END AS recommended_limit,

    -- Transparência da decisão
    CONCAT(
        'ever_default=', ever_default,
        ', overdraft_risk=', overdraft_risk,
        ', avg_net_cashflow=', avg_net_cashflow,
        ', balance_profile=', balance_profile
    ) AS rationale_features

FROM workspace.gold.credit_risk_segmented;

num_affected_rows,num_inserted_rows
